In [118]:
from requests import get
from bs4 import BeautifulSoup
from dataclasses import dataclass
import json
from typing import TypedDict
import os
import time
from typing import Tuple

In [119]:
class Joueur(TypedDict):
    rank: str
    pays: str
    lien_joueur: str
    nom_joueur: str
    pays_abreviation: str
    age: str
    points: str

In [120]:
current_dir: str = os.getcwd()

parent_dir: str = os.path.dirname(current_dir)

file_path: str = os.path.join(parent_dir, "donnees", "joueurs.json")

# Charger le JSON
with open(file_path, "r") as fichier:
    joueurs:list[dict] = json.load(fichier)
    assert len(joueurs) == 900

In [121]:
parent_dir

'd:\\projet\\ml-webscrap-tennis'

In [122]:
joueurs[0]

{'rank': '1.',
 'pays': 'Italy',
 'lien_joueur': 'https://www.tennisendirect.net/atp/jannik-sinner/',
 'nom_joueur': 'Jannik Sinner',
 'pays_abreviation': 'ITA',
 'age': '23 ans',
 'points': '11330'}

In [123]:
print(joueurs[0]['nom_joueur'])

Jannik Sinner


In [124]:
print(joueurs[0]['lien_joueur'])

https://www.tennisendirect.net/atp/jannik-sinner/


In [125]:
liens_joueurs = (joueurs[i]['lien_joueur'] for i in range(len(joueurs)))

In [126]:
liens_joueurs

<generator object <genexpr> at 0x0000017C205A4BA0>

# Test de récupération des stats d'un joueur (Sinner)

In [127]:
Sinner = "https://www.tennisendirect.net/atp/jannik-sinner/"

In [128]:
reponse = get(Sinner)
print(Sinner, reponse.status_code)
assert reponse.status_code == 200

https://www.tennisendirect.net/atp/jannik-sinner/ 200


In [129]:
detail_Sinner = BeautifulSoup(reponse.text, features="lxml")

In [130]:
profil = detail_Sinner.find_all("div", attrs={"class": "player_stats"})
assert len(profil) == 1
statistiques = detail_Sinner.find_all("table", attrs={"class": "table_stats"})
assert len(statistiques) == 1
*_, derniers_match = detail_Sinner.find_all("table", attrs={"class": "table_pmatches"})

In [131]:
type(statistiques)

bs4.element.ResultSet

In [132]:
print("""profil: {profil} \n stats: {statistiques} \n matchs: {derniers_match}""".format(
    profil=profil, statistiques=statistiques, derniers_match=derniers_match))

profil: [<div class="player_stats">
            Nom: <b><a href="https://www.tennisendirect.net/atp/jannik-sinner/" title="Jannik Sinner">Jannik Sinner</a></b><br/>            Pays: <b>Italy</b><br/>            Date de naissance: <b>16.08.01, 23 ans</b><br/> <a href="https://www.tennisendirect.net/atp/classement/" title="Position dans le classement">Classement ATP</a>: <b>1</b><br/>            TOP position dans le classement: <b>1</b> (02.12.24, 11830 points)<br/>            Points: <b>11830</b><br/>            Primes: <b>17043434 $</b><br/>            Total de matchs: <b>491</b><br/>            Victoires: <b>356</b><br/>            Taux de réussite: <b>72.51 %</b><br/>
</div>] 
 stats: [<table class="table_stats">
<tr class="header"><td>année</td><td>sommaire</td><td><a href="?su=1" rel="nofollow" title="surface: dure">dure</a></td><td><a href="?su=2" rel="nofollow" title="surface: terre battue">terre battue</a></td><td><a href="?su=3" rel="nofollow" title="surface: salle">salle</a></

In [133]:
@dataclass
class Profil:
    nom: str
    pays: str
    date_naissance: str
    age: str
    classement_atp: str
    points: str
    primes: str
    total_match: str
    victoires: str
    taux_reussite: str

In [134]:
@dataclass
class Statistiques:
    annee: str
    sommaire: str
    dure: str
    terre_battue: str
    salle: str
    carpet: str
    gazon: str
    acryl: str

In [158]:
@dataclass
class Matchs:
    date: str
    stage: str
    nom_joueur: str
    nom_opposant: str
    score: str
    resultat: str
    lien_detail_match: str
    tournoi: str
    type_terrain :str

## Profil 

In [136]:
print(profil)

[<div class="player_stats">
            Nom: <b><a href="https://www.tennisendirect.net/atp/jannik-sinner/" title="Jannik Sinner">Jannik Sinner</a></b><br/>            Pays: <b>Italy</b><br/>            Date de naissance: <b>16.08.01, 23 ans</b><br/> <a href="https://www.tennisendirect.net/atp/classement/" title="Position dans le classement">Classement ATP</a>: <b>1</b><br/>            TOP position dans le classement: <b>1</b> (02.12.24, 11830 points)<br/>            Points: <b>11830</b><br/>            Primes: <b>17043434 $</b><br/>            Total de matchs: <b>491</b><br/>            Victoires: <b>356</b><br/>            Taux de réussite: <b>72.51 %</b><br/>
</div>]


In [137]:
def genere_profil(profil):
    """
    Génère un profil de joueur de tennis à partir des informations HTML fournies.

    Args:
        profil (list): Une liste d'éléments HTML, où le premier élément contient les informations du joueur.

    Returns:
        Profil: Un objet contenant les informations du joueur, telles que le nom, le pays, 
        la date de naissance, l'âge, le classement ATP, les points, les primes, le total de matchs, 
        les victoires, et le taux de réussite.

    Remarque:
        En cas d'erreur d'extraction (par exemple, si les balises ne sont pas présentes ou sont mal formées),
        des valeurs par défaut ("NA") sont utilisées pour les champs.
    """
    div = profil[0]

    try:
        nom_joueur = div.find("a").text.strip()
        pays = div.find_all("b")[1].text.strip()
        date_naissance = div.find_all("b")[2].text.strip().split(", ")[0]
        age = div.find_all("b")[2].text.strip().split(", ")[1]
        classement_atp = div.find_all("b")[3].text.strip()
        points = div.find_all("b")[5].text.strip()
        primes = div.find_all("b")[6].text.strip()
        total_match = div.find_all("b")[7].text.strip()
        victoires = div.find_all("b")[8].text.strip()
        taux_reussite = div.find_all("b")[9].text.strip()
    
    except (IndexError, AttributeError):
        nom_joueur, pays, date_naissance, age, classement_atp, points, primes, total_match, victoires, taux_reussite =(
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
        
    return Profil(
        nom=nom_joueur,
        pays=pays,
        date_naissance=date_naissance,
        age=age,
        classement_atp=classement_atp,
        points=points,
        primes=primes,
        total_match=total_match,
        victoires=victoires,
        taux_reussite=taux_reussite,
    )  


In [138]:
Sinner_profil = genere_profil(profil)
Sinner_profil

Profil(nom='Jannik Sinner', pays='Italy', date_naissance='16.08.01', age='23 ans', classement_atp='1', points='11830', primes='17043434 $', total_match='491', victoires='356', taux_reussite='72.51 %')

## Statistiques

In [139]:
def extraire_lignes(table):
    """
    Extrait les lignes d'une table HTML ayant des classes spécifiques.

    Args:
        table (BeautifulSoup): Objet représentant une table HTML.

    Returns:
        list: Liste des éléments `<tr>` avec les classes "pair" ou "unpair".
    """
    return table.find_all(
        "tr", class_=lambda class_name: class_name in ["pair", "unpair"]
    )

In [140]:
def genere_statistiques(ligne):
    """
    Génère des statistiques annuelles d'un joueur à partir d'une ligne HTML.

    Args:
        ligne (BeautifulSoup): Élément HTML représentant une ligne `<tr>` contenant les statistiques.

    Returns:
        Statistiques: Objet contenant les statistiques suivantes :
            - annee (str): Année des statistiques.
            - sommaire (str): Résumé des performances.
            - dure (str): Performances sur surface dure.
            - terre_battue (str): Performances sur terre battue.
            - salle (str): Performances en salle.
            - carpet (str): Performances sur moquette.
            - gazon (str): Performances sur gazon.
            - acryl (str): Performances sur acrylique.

    Raises:
        None: Retourne `None` si le format de la ligne est inattendu.
    """

    colonnes = [td.text.strip() for td in ligne.find_all("td")]
    
    if len(colonnes) == 8:
        annee, sommaire, dure, terre_battue, salle, carpet, gazon, acryl = colonnes
    else:
        print("Format inattendu dans la ligne:", colonnes)
        return None

    return Statistiques(
        annee = annee,
        sommaire = sommaire,
        dure = dure,
        terre_battue = terre_battue,
        salle = salle,
        carpet = carpet,
        gazon = gazon,
        acryl = acryl,
    ) 

In [141]:
lignes_stats = extraire_lignes(statistiques[0])
Sinner_statistiques = [genere_statistiques(ligne) for ligne in lignes_stats if genere_statistiques(ligne)]
Sinner_statistiques

[Statistiques(annee='2024', sommaire='74/7', dure='41/3', terre_battue='11/3', salle='13/0', carpet='0/0', gazon='9/1', acryl='0/0'),
 Statistiques(annee='2023', sommaire='66/18', dure='29/7', terre_battue='8/4', salle='21/4', carpet='0/0', gazon='8/3', acryl='0/0'),
 Statistiques(annee='2022', sommaire='47/17', dure='21/7', terre_battue='15/4', salle='7/4', carpet='0/0', gazon='4/2', acryl='0/0'),
 Statistiques(annee='2021', sommaire='50/24', dure='23/10', terre_battue='10/6', salle='17/6', carpet='0/0', gazon='0/2', acryl='0/0'),
 Statistiques(annee='2020', sommaire='24/16', dure='2/6', terre_battue='10/5', salle='12/5', carpet='0/0', gazon='0/0', acryl='0/0'),
 Statistiques(annee='2019', sommaire='62/24', dure='12/6', terre_battue='17/8', salle='31/7', carpet='0/0', gazon='2/3', acryl='0/0'),
 Statistiques(annee='2018', sommaire='31/23', dure='13/9', terre_battue='17/12', salle='1/1', carpet='0/1', gazon='0/0', acryl='0/0'),
 Statistiques(annee='2017', sommaire='1/2', dure='0/0', te

## Matchs

In [159]:
def filtre_tour_head(ligne) -> Tuple[Matchs, str, str]:
    """
    Filtre les informations d'une ligne HTML pour extraire les données d'un match et des détails du tournoi.

    Args:
        ligne (BeautifulSoup): Élément HTML représentant une ligne `<tr>` contenant les informations du match.

    Returns:
        Tuple[Matchs, str, str]: 
            - Un objet `Matchs` avec les détails du match (date, stage, vainqueur, perdant, score, etc.).
            - Le nom du tournoi (str).
            - Le type de terrain (str).
    """

    (
        date, stage, _, _, score, _, _, _, type_terrain
    ) = [td.text.strip() for td in ligne.find_all("td")]

    try:
        nom_joueur = ligne.find_all("b")[0].text.strip()
        nom_opposant = ligne.find_all("a")[0].text.strip()
        resultat = ligne.find_all("img")[0]["alt"]
        lien_detail_match = ligne.find_all("a")[1]["href"]
        tournoi = ligne.find_all("a")[2]["title"]
    
    except (IndexError, AttributeError):
        nom_joueur, nom_opposant, resultat, lien_detail_match, tournoi =(
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
        
    return Matchs(
        date=date,
        stage=stage,
        nom_joueur=nom_joueur,
        nom_opposant=nom_opposant,
        score=score,
        resultat=resultat,
        lien_detail_match=lien_detail_match,
        tournoi=tournoi,
        type_terrain=type_terrain,
    ), tournoi, type_terrain

In [160]:
def filtre_no_tour_head(ligne) -> tuple:
    """
    Extrait les informations d'une ligne HTML pour un match sans détails sur le tournoi.

    Args:
        ligne (BeautifulSoup): Élément HTML représentant une ligne `<tr>` contenant les informations du match.

    Returns:
        tuple: Une tuple contenant :
            - date (str): Date du match.
            - stage (str): Phase du tournoi (par exemple, finale, demi-finale).
            - nom_vainqueur (str): Nom du joueur vainqueur.
            - nom_opposant (str): Nom du joueur perdant.
            - score (str): Score du match.
            - resultat (str): Résultat du match (par exemple, "victoire" ou "défaite").
            - lien_detail_match (str): Lien vers les détails du match.
    """
    
    (
        date, stage, _, _, score, _, _
    ) = [td.text.strip() for td in ligne.find_all("td")]

    try:
        nom_joueur = ligne.find_all("b")[0].text.strip()
        nom_opposant = ligne.find_all("a")[0].text.strip()
        resultat = ligne.find_all("img")[0]["alt"]
        lien_detail_match = ligne.find_all("a")[1]["href"]
        
    except (IndexError, AttributeError):
        nom_joueur, nom_opposant, resultat, lien_detail_match =(
            "NA",
            "NA",
            "NA",
            "NA",
        )
        
    return date, stage, nom_joueur, nom_opposant, score, resultat, lien_detail_match

In [161]:
def genere_derniers_matchs(ligne_derniers_matchs):
    """
    Génère une liste des derniers matchs à partir des lignes HTML d'une table.

    Args:
        ligne_derniers_matchs (list): Liste d'éléments HTML représentant les lignes `<tr>` d'une table de matchs.

    Returns:
        list: Une liste d'objets `Matchs`, chacun représentant un match avec les informations suivantes :
            - date (str): Date du match.
            - stage (str): Phase du tournoi (par exemple, finale, demi-finale).
            - nom_joueur (str): Nom du joueur vainqueur.
            - nom_perdant (str): Nom du joueur perdant.
            - score (str): Score du match.
            - resultat (str): Résultat du match (par exemple, victoire, défaite).
            - lien_detail_match (str): Lien vers les détails du match.
            - tournoi (str): Nom du tournoi associé au match.
            - type_terrain (str): Type de terrain sur lequel le match a été joué.
    """

    tournoi_precedent = ""
    type_terrain_precedent = ""
    
    matchs = []
    for ligne in ligne_derniers_matchs:
        if "tour_head" in ligne["class"]:
            match, tournoi_precedent, type_terrain_precedent = filtre_tour_head(ligne)
            matchs.append(match)
            
        elif "pair" in ligne["class"] or "unpair" in ligne["class"]:
            date, stage, nom_joueur, nom_opposant, score, resultat, lien_detail_match = filtre_no_tour_head(ligne)

            tournoi = tournoi_precedent
            type_terrain = type_terrain_precedent

            match = Matchs(
                date=date,
                stage=stage,
                nom_joueur=nom_joueur,
                nom_opposant=nom_opposant,
                score=score,
                resultat=resultat,
                lien_detail_match=lien_detail_match,
                tournoi=tournoi,
                type_terrain=type_terrain
            )
            matchs.append(match)
            
    return matchs

In [162]:
ligne_derniers_matchs = extraire_lignes(derniers_match)
Sinner_derniers_matchs = genere_derniers_matchs(ligne_derniers_matchs)
Sinner_derniers_matchs

[Matchs(date='24.11.24', stage='', nom_joueur='Jannik Sinner', nom_opposant='Tallon Griekspoor', score='7-62, 6-2', resultat='victoire', lien_detail_match='https://www.tennisendirect.net/atp/match/jannik-sinner-VS-tallon-griekspoor/davis-cup-world-group-f-ita-ned-2024/', tournoi='Davis Cup, World Group, F, ITA-NED 2-0 / ', type_terrain='salle'),
 Matchs(date='23.11.24', stage='', nom_joueur='Jannik Sinner', nom_opposant='Alex De Minaur', score='6-3, 6-4', resultat='victoire', lien_detail_match='https://www.tennisendirect.net/atp/match/jannik-sinner-VS-alex-de-minaur/davis-cup-world-group-sf-ita-aus-2024/', tournoi='Davis Cup, World Group, SF, ITA-AUS 2-0 / ', type_terrain='salle'),
 Matchs(date='21.11.24', stage='', nom_joueur='Jannik Sinner', nom_opposant='Sebastian Baez', score='6-2, 6-1', resultat='victoire', lien_detail_match='https://www.tennisendirect.net/atp/match/jannik-sinner-VS-sebastian-baez/davis-cup-world-group-qf-ita-arg-2024/', tournoi='Davis Cup, World Group, QF, ITA-AR

## RESULTAT

In [146]:
Sinner_profil

Profil(nom='Jannik Sinner', pays='Italy', date_naissance='16.08.01', age='23 ans', classement_atp='1', points='11830', primes='17043434 $', total_match='491', victoires='356', taux_reussite='72.51 %')

In [147]:
Sinner_statistiques

[Statistiques(annee='2024', sommaire='74/7', dure='41/3', terre_battue='11/3', salle='13/0', carpet='0/0', gazon='9/1', acryl='0/0'),
 Statistiques(annee='2023', sommaire='66/18', dure='29/7', terre_battue='8/4', salle='21/4', carpet='0/0', gazon='8/3', acryl='0/0'),
 Statistiques(annee='2022', sommaire='47/17', dure='21/7', terre_battue='15/4', salle='7/4', carpet='0/0', gazon='4/2', acryl='0/0'),
 Statistiques(annee='2021', sommaire='50/24', dure='23/10', terre_battue='10/6', salle='17/6', carpet='0/0', gazon='0/2', acryl='0/0'),
 Statistiques(annee='2020', sommaire='24/16', dure='2/6', terre_battue='10/5', salle='12/5', carpet='0/0', gazon='0/0', acryl='0/0'),
 Statistiques(annee='2019', sommaire='62/24', dure='12/6', terre_battue='17/8', salle='31/7', carpet='0/0', gazon='2/3', acryl='0/0'),
 Statistiques(annee='2018', sommaire='31/23', dure='13/9', terre_battue='17/12', salle='1/1', carpet='0/1', gazon='0/0', acryl='0/0'),
 Statistiques(annee='2017', sommaire='1/2', dure='0/0', te

In [148]:
Sinner_derniers_matchs

[Matchs(date='24.11.24', stage='', nom_vainqueur='Jannik Sinner', nom_opposant='Tallon Griekspoor', score='7-62, 6-2', resultat='victoire', lien_detail='https://www.tennisendirect.net/atp/match/jannik-sinner-VS-tallon-griekspoor/davis-cup-world-group-f-ita-ned-2024/', tournoi='Davis Cup, World Group, F, ITA-NED 2-0 / ', type_terrain='salle'),
 Matchs(date='23.11.24', stage='', nom_vainqueur='Jannik Sinner', nom_opposant='Alex De Minaur', score='6-3, 6-4', resultat='victoire', lien_detail='https://www.tennisendirect.net/atp/match/jannik-sinner-VS-alex-de-minaur/davis-cup-world-group-sf-ita-aus-2024/', tournoi='Davis Cup, World Group, SF, ITA-AUS 2-0 / ', type_terrain='salle'),
 Matchs(date='21.11.24', stage='', nom_vainqueur='Jannik Sinner', nom_opposant='Sebastian Baez', score='6-2, 6-1', resultat='victoire', lien_detail='https://www.tennisendirect.net/atp/match/jannik-sinner-VS-sebastian-baez/davis-cup-world-group-qf-ita-arg-2024/', tournoi='Davis Cup, World Group, QF, ITA-ARG 2-1 / '

### Ecriture des résultats 

In [99]:
Sinner_profil.__dict__

{'nom': 'Jannik Sinner',
 'pays': 'Italy',
 'date_naissance': '16.08.01',
 'age': '23 ans',
 'classement_atp': '1',
 'points': '11830',
 'primes': '17043434 $',
 'total_match': '491',
 'victoires': '356',
 'taux_reussite': '72.51 %'}

In [100]:
stats_test = [stats.__dict__ for stats in Sinner_statistiques]
stats_test

[{'annee': '2024',
  'sommaire': '74/7',
  'dure': '41/3',
  'terre_battue': '11/3',
  'salle': '13/0',
  'carpet': '0/0',
  'gazon': '9/1',
  'acryl': '0/0'},
 {'annee': '2023',
  'sommaire': '66/18',
  'dure': '29/7',
  'terre_battue': '8/4',
  'salle': '21/4',
  'carpet': '0/0',
  'gazon': '8/3',
  'acryl': '0/0'},
 {'annee': '2022',
  'sommaire': '47/17',
  'dure': '21/7',
  'terre_battue': '15/4',
  'salle': '7/4',
  'carpet': '0/0',
  'gazon': '4/2',
  'acryl': '0/0'},
 {'annee': '2021',
  'sommaire': '50/24',
  'dure': '23/10',
  'terre_battue': '10/6',
  'salle': '17/6',
  'carpet': '0/0',
  'gazon': '0/2',
  'acryl': '0/0'},
 {'annee': '2020',
  'sommaire': '24/16',
  'dure': '2/6',
  'terre_battue': '10/5',
  'salle': '12/5',
  'carpet': '0/0',
  'gazon': '0/0',
  'acryl': '0/0'},
 {'annee': '2019',
  'sommaire': '62/24',
  'dure': '12/6',
  'terre_battue': '17/8',
  'salle': '31/7',
  'carpet': '0/0',
  'gazon': '2/3',
  'acryl': '0/0'},
 {'annee': '2018',
  'sommaire': '31/

In [101]:
os.pardir

'..'

In [102]:
current_dir: str = os.getcwd()

parent_dir: str = os.path.dirname(current_dir)

In [103]:
file_path_profil: str = os.path.join(parent_dir, "donnees", "profil_sinner.json")
with open(file_path_profil, "w") as fichier:
    fichier.write(
        json.dumps(
            [Sinner_profil.__dict__], ensure_ascii=False, indent=4
        )
    )

In [104]:
file_path_statistiques: str = os.path.join(parent_dir, "donnees", "stats_sinner.json")
with open(file_path_statistiques, "w") as fichier:
    fichier.write(
        json.dumps(
            [stats.__dict__ for stats in Sinner_statistiques], ensure_ascii=False, indent=4
        )
    )

In [105]:
file_path_matchs: str = os.path.join(parent_dir, "donnees", "matchs_sinner.json")
with open(file_path_matchs, "w") as fichier:
    fichier.write(
        json.dumps(
            [match.__dict__ for match in Sinner_derniers_matchs], ensure_ascii=False, indent=4
        )
    )

In [163]:
joueurs_data = {}

joueurs_data[Sinner_profil.nom] = {
    "profil": Sinner_profil.__dict__,
    "statistiques": [stats.__dict__ for stats in Sinner_statistiques],
    "matchs": [match.__dict__ for match in Sinner_derniers_matchs]
}

file_path_joueurs: str = os.path.join(parent_dir, "donnees", "joueurs_complets.json")

if os.path.exists(file_path_joueurs):
    with open(file_path_joueurs, "r") as fichier:
        try:
            joueurs_data.update(json.load(fichier))
        except json.JSONDecodeError:
            print("Fichier JSON corrompu ou vide. Il sera remplacé.")

with open(file_path_joueurs, "w", encoding= "utf-8") as fichier:
    json.dump(joueurs_data, fichier, ensure_ascii=False, indent=4)


# Récupération des stats de tous les joueurs

In [37]:
def get_page_joueur(liste_lien_joueur):
    for lien in liste_lien_joueur:
        reponse = get(lien)
        print(lien, reponse.status_code)
        assert reponse.status_code == 200
        time.sleep(2)

In [38]:
get_page_joueur(liens_joueurs)

https://www.tennisendirect.net/atp/jannik-sinner/ 200


KeyboardInterrupt: 